In [1]:
import pandas as pd
import pickle

from pymongo import MongoClient

import spotipy
from config import api_access_token

### Connecting to MongoDB

In [4]:
client = MongoClient()
db = client.spotify
db.list_collection_names()

['artists', 'albums', 'track_features', 'tracks']

In [5]:
cursor = db.tracks.find(
    {}, {'_id': 0, 'album_uri': 1, 'artist_id': 1, 'artist_name': 1,
         'artist_uri': 1, 'duration_ms': 1, 'explicit': 1, 'id': 1,
         'name': 1, 'track_number': 1, 'uri': 1, 'song_spotify_page': 1})
df = pd.DataFrame(list(cursor))
df.head()

,album_uri,artist_id,artist_name,artist_uri,duration_ms,explicit,id,name,song_spotify_page,track_number,uri
0,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,138626,False,4okEZakOVppAtP4Dawd52x,Marry Me,https://open.spotify.com/track/4okEZakOVppAtP4...,1,spotify:track:4okEZakOVppAtP4Dawd52x
1,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,187106,False,2LV6sB5zTsu0R5r5kWohlD,Loop Duplicate My Heart,https://open.spotify.com/track/2LV6sB5zTsu0R5r...,2,spotify:track:2LV6sB5zTsu0R5r5kWohlD
2,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,192000,False,2RPcqvy4bdznms2eHU6vnE,A Couple Of Instruments,https://open.spotify.com/track/2RPcqvy4bdznms2...,3,spotify:track:2RPcqvy4bdznms2eHU6vnE
3,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,176026,False,53uzYuKe433aXBjzBiuvqe,Parakit,https://open.spotify.com/track/53uzYuKe433aXBj...,4,spotify:track:53uzYuKe433aXBjzBiuvqe
4,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,198013,False,7p0pJgizlHS5msrgDPU6li,Trees And Squirrels,https://open.spotify.com/track/7p0pJgizlHS5msr...,5,spotify:track:7p0pJgizlHS5msrgDPU6li


## Removing non-queried artists

In [6]:
returned_artists = df.artist_name.unique().tolist()

In [7]:
pickle_in = open('Data/indie_pop_artists', 'rb')
queried_artists = pickle.load(pickle_in)
pickle_in.close()

Again, we check to make sure that we're only looking at the artists that were both queried and returned.

In [8]:
print(len(queried_artists))
print(len(returned_artists))
print(len(set(queried_artists).intersection(set(returned_artists))))

716
714
539


In [9]:
artists_df = (pd.DataFrame(list(set(queried_artists)
                                .intersection(set(returned_artists)))))
artists_df.columns = ['artist_name']
artists_df.head()

,artist_name
0,Melanie Martinez
1,Il Genio
2,Sarah P.
3,Zella Day
4,The Submarines


In [10]:
df = df.merge(artists_df, on='artist_name')
df.head()

,album_uri,artist_id,artist_name,artist_uri,duration_ms,explicit,id,name,song_spotify_page,track_number,uri
0,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,138626,False,4okEZakOVppAtP4Dawd52x,Marry Me,https://open.spotify.com/track/4okEZakOVppAtP4...,1,spotify:track:4okEZakOVppAtP4Dawd52x
1,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,187106,False,2LV6sB5zTsu0R5r5kWohlD,Loop Duplicate My Heart,https://open.spotify.com/track/2LV6sB5zTsu0R5r...,2,spotify:track:2LV6sB5zTsu0R5r5kWohlD
2,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,192000,False,2RPcqvy4bdznms2eHU6vnE,A Couple Of Instruments,https://open.spotify.com/track/2RPcqvy4bdznms2...,3,spotify:track:2RPcqvy4bdznms2eHU6vnE
3,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,176026,False,53uzYuKe433aXBjzBiuvqe,Parakit,https://open.spotify.com/track/53uzYuKe433aXBj...,4,spotify:track:53uzYuKe433aXBjzBiuvqe
4,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,198013,False,7p0pJgizlHS5msrgDPU6li,Trees And Squirrels,https://open.spotify.com/track/7p0pJgizlHS5msr...,5,spotify:track:7p0pJgizlHS5msrgDPU6li


Below, we can see the number of songs that we have for unique artist/album combinations that were both returned and queried.

In [11]:
df.shape

(35538, 11)

## Get Track Features

In [31]:
def get_track_features(album_uri_s_list, df, access_token):
    """
    Takes in a list of album URIs, a DataFrame containing all of our song information,
    and an access token.
    
    Queries Spotify and returns track features.
    """

    sp = spotipy.Spotify(auth=access_token)

    for album_uri in album_uri_s_list[2:]:
        track_info = df[df.album_uri == album_uri][['artist_uri', 'uri']]

        # Arbitrarily takes the first artist_uri. They are all identical.
        artist_uri = track_info.artist_uri.values[0]

        track_uri_s_list = track_info.uri.values.tolist()
        # there's a hard limit of 50 queires so we limit the list size to that
        if len(track_uri_s_list) > 50:
            track_uri_s_list = track_uri_s_list[:50]
        else:
            pass
        track_features_list = sp.audio_features(track_uri_s_list)

        save_features_to_mongoDB(track_features_list, artist_uri, album_uri)

In [25]:
def save_features_to_mongoDB(track_features_list, artist_uri, album_uri):
    """
    Takes track features, artist URI, and album URI as input.
    
    Saves the track features and the artist and album URIs for later merging to other datasets.
    """    

    for track_features in track_features_list:
        track_features['album_uri'] = album_uri
        track_features['artist_uri'] = artist_uri
        db.track_features.insert_one(track_features)

In [19]:
access_token = api_access_token

In [17]:
album_uri_s_list = df.album_uri.unique().tolist()
len(album_uri_s_list)

2645

In [32]:
get_track_features(album_uri_s_list, df, access_token)